In [1]:
# Imports:
import pandas as pd
import torch

In [2]:
# Load TE-for-Event-Extraction model:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("veronica320/TE-for-Event-Extraction")
model = AutoModelForSequenceClassification.from_pretrained("veronica320/TE-for-Event-Extraction")

In [3]:
# Load the data:
df = pd.read_csv('Combined_SPA_Callback.csv')
df

,Date,ID,Text
0,26-Jul-17,1.010040e+11,please call daughter Dianne Thomas 07920 07565...
1,28-Jul-17,1.010040e+11,please speak to sister - Alyson Powell on abpv...
2,28-Jul-17,1.010040e+11,KELLY HAS RUNG - HER PARENTS ARE DUE TO HAVE W...
3,31-Jul-17,1.010040e+11,"Mark Hitchings, Scheme Manager Swn Yr Afon con..."
4,28-Jul-17,1.010040e+11,SON BRIAN HASFORD RUN G- HE IS RESIDENT IN AUS...
...,...,...,...
11353,13-Jun-23,1.010000e+11,"Enquiring about a downstairs toilet, states he..."
11354,13-Jun-23,1.010000e+11,Phoning on behalf on her mother she has no fee...
11355,13-Jun-23,1.010000e+11,States on saturday they had to call an ambulan...
11356,13-Jun-23,1.010000e+11,Nadia Obaji daughter of Abdel Obaji contacted ...


In [4]:
# This cell takes ~ 2 hours to run:

# Combining the 2 Events to input into model:
first_text = list(df["Text"])
second_text = "Urgent"
combined_text = []
for text in first_text:
    text = text + ". " + second_text
    combined_text.append(text)

# Chunking the data into smaller batches:
batch_size = 256
batch_texts = []
for i in range(0, len(combined_text), batch_size):
    batch_texts.append(combined_text[i : i + batch_size])

logits = []
counter = 0
for input_text in batch_texts:
    counter += len(input_text)
    # Tokenizing the text:
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=150)
    # Running inputs through the model:
    with torch.no_grad():
        batch_outputs = model(**inputs)
    logits.append(batch_outputs.logits)
    print("{}/{} inputs done.".format(counter,len(first_text)),end='\r')

# Concatenate lists of logits together:
outputs = {"logits":torch.cat(logits)}

In [5]:
# Get the probabilities for all classes:
probabilities = torch.softmax(outputs["logits"], dim=1).tolist()

# Get the top result:
top_results = torch.argmax(outputs["logits"], dim=1).tolist()

# Create a dictionary to map the labels:
label_map = {0: 'Contradiction', 1: 'Neutral', 2: 'Entailment'}

# Round up the results:
rounded_probs = []
for i in range(len(combined_text)):
    rounded_probs.append([round(prob, 4) for prob in probabilities[i]])

# Combine the predicted label and probabilities into a dictionary:
results_list=[]
for i in range(len(combined_text)):
    results_list.append({label_map[j]: prob for j, prob in enumerate(rounded_probs[i])})
    results_list[i]["Result"] = label_map[top_results[i]]

# Print the result as a dataframe:
results_df = pd.DataFrame(results_list)
results_df

,Contradiction,Neutral,Entailment,Result
0,0.9581,0.0364,0.0055,Contradiction
1,0.9999,0.0000,0.0001,Contradiction
2,0.9998,0.0000,0.0002,Contradiction
3,0.9976,0.0004,0.0021,Contradiction
4,0.9996,0.0001,0.0004,Contradiction
...,...,...,...,...
11353,0.9996,0.0001,0.0003,Contradiction
11354,0.9891,0.0068,0.0040,Contradiction
11355,0.9996,0.0002,0.0001,Contradiction
11356,0.9998,0.0001,0.0001,Contradiction


In [6]:
final_df = pd.concat([df,results_df], axis=1)
final_df.to_csv("Model_Results.csv", index=False)
final_df

,Date,ID,Text,Contradiction,Neutral,Entailment,Result
0,26-Jul-17,1.010040e+11,please call daughter Dianne Thomas 07920 07565...,0.9581,0.0364,0.0055,Contradiction
1,28-Jul-17,1.010040e+11,please speak to sister - Alyson Powell on abpv...,0.9999,0.0000,0.0001,Contradiction
2,28-Jul-17,1.010040e+11,KELLY HAS RUNG - HER PARENTS ARE DUE TO HAVE W...,0.9998,0.0000,0.0002,Contradiction
3,31-Jul-17,1.010040e+11,"Mark Hitchings, Scheme Manager Swn Yr Afon con...",0.9976,0.0004,0.0021,Contradiction
4,28-Jul-17,1.010040e+11,SON BRIAN HASFORD RUN G- HE IS RESIDENT IN AUS...,0.9996,0.0001,0.0004,Contradiction
...,...,...,...,...,...,...,...
11353,13-Jun-23,1.010000e+11,"Enquiring about a downstairs toilet, states he...",0.9996,0.0001,0.0003,Contradiction
11354,13-Jun-23,1.010000e+11,Phoning on behalf on her mother she has no fee...,0.9891,0.0068,0.0040,Contradiction
11355,13-Jun-23,1.010000e+11,States on saturday they had to call an ambulan...,0.9996,0.0002,0.0001,Contradiction
11356,13-Jun-23,1.010000e+11,Nadia Obaji daughter of Abdel Obaji contacted ...,0.9998,0.0001,0.0001,Contradiction
